In [ ]:
import os, re
import random, math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action='ignore')



# 데이터 로드할 때 빠르게 로드할 수 있도록하는 설정 변수
AUTOTUNE = tf.data.experimental.AUTOTUNE
# X-RAY 이미지 사이즈 변수
IMAGE_SIZE = [180, 180]


# 데이터 경로 변수
ROOT_PATH = os.path.join(os.getenv('HOME'), 'aiffel')
TRAIN_PATH = ROOT_PATH + '/chest_xray/data/train/*/*' # *은 모든 디렉토리와 파일을 의미합니다.
VAL_PATH = ROOT_PATH + '/chest_xray/data/val/*/*'
TEST_PATH = ROOT_PATH + '/chest_xray/data/test/*/*'



BATCH_SIZE = 16  # 16  ->  16
EPOCHS     = 6  #  5  ->   6


train_filenames = tf.io.gfile.glob(TRAIN_PATH)
test_filenames = tf.io.gfile.glob(TEST_PATH)
val_filenames = tf.io.gfile.glob(VAL_PATH)


# train 데이터와 validation 데이터를 모두 filenames에 담습니다.
filenames = tf.io.gfile.glob(TRAIN_PATH)
filenames.extend(tf.io.gfile.glob(VAL_PATH))

# 모인 filenames를 8:2로 나눕니다
train_size = math.floor(len(filenames)*0.8)
random.seed(8)
random.shuffle(filenames)
train_filenames = filenames[:train_size]
val_filenames = filenames[train_size:]
print(len(train_filenames))
print(len(val_filenames))

COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])


train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)


TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()


def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return tf.cast(parts[-2] == "PNEUMONIA", tf.int32)   # Convert boolean to integer (1 for PNEUMONIA, 0 for NORMAL)


def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, IMAGE_SIZE)
  return img


# 이미지 파일의 경로를 입력하면 이미지와 라벨을 읽어옵니다.
def process_path(file_path):
    label = get_label(file_path) # 라벨 검출
    img = tf.io.read_file(file_path) # 이미지 읽기
    img = decode_img(img) # 이미지를 알맞은 형식으로 수정
    return img, label


train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

test_list_ds = tf.data.Dataset.list_files(TEST_PATH)
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)


# 데이터 좌우 반전 후 이미지와 레이블 반환 함수 
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label


def prepare_for_training(ds, shuffle_buffer_size=1000):
    # 데이터 증강: 좌우 반전한 이미지 추가
    ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)


weight_for_0 = (1 / COUNT_NORMAL) * (TRAIN_IMG_COUNT) / 2.0
weight_for_1 = (1 / COUNT_PNEUMONIA) * (TRAIN_IMG_COUNT) / 2.0


class_weight = {0: weight_for_0, 1: weight_for_1}


def identity_block(x, filters):
    x_skip = x
    f1, f2 = filters

    # First Conv
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Second Conv
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Third Conv
    x = tf.keras.layers.Conv2D(filters=f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Identity + Feature
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.ReLU()(x)
    return x

def convolutional_block(x, filters, strides=(2, 2)):
    x_skip = x
    f1, f2 = filters

    # First Conv
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=(1, 1), strides=strides, padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Second Conv
    x = tf.keras.layers.Conv2D(filters=f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    # Third Conv
    x = tf.keras.layers.Conv2D(filters=f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Shortcut Conv
    x_skip = tf.keras.layers.Conv2D(filters=f2, kernel_size=(1, 1), strides=strides, padding='valid')(x_skip)
    x_skip = tf.keras.layers.BatchNormalization()(x_skip)

    # Shorcut Conv + Feature
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.ReLU()(x)
    return x

def build_resnet_18():
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    
    # Conv1
    x = tf.keras.layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Conv2_x
    x = convolutional_block(x, filters=(64, 256), strides=(1, 1))
    x = identity_block(x, filters=(64, 256))
    x = identity_block(x, filters=(64, 256))
    x = identity_block(x, filters=(64, 256))

    # conv3_x
    x = convolutional_block(x, filters=(128, 512), strides=(2, 2))
    x = identity_block(x, filters=(128, 512))
    x = identity_block(x, filters=(128, 512))
    x = identity_block(x, filters=(128, 512))

    # Conv4_x
    x = convolutional_block(x, filters=(256, 1024), strides=(2, 2))
    x = identity_block(x, filters=(256, 1024))
    x = identity_block(x, filters=(256, 1024))
    x = identity_block(x, filters=(256, 1024))
    x = identity_block(x, filters=(256, 1024))
    x = identity_block(x, filters=(256, 1024))

    # Conv5_x
    x = convolutional_block(x, filters=(512, 2048), strides=(2, 2))
    x = identity_block(x, filters=(512, 2048))
    x = identity_block(x, filters=(512, 2048))

    # Average pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # FC
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

model = build_resnet_18()

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS
)

history = model.fit(
    train_ds,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_ds,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight
)

fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

loss, accuracy, precision, recall = model.evaluate(test_ds)
print(f'Loss: {loss},\nAccuracy: {accuracy},\nPrecision: {precision},\nRecall: {recall}')


4185
1047
Epoch 1/6
261/261 [==============================] - 69s 217ms/step - loss: 0.4264 - accuracy: 0.8575 - precision: 0.9432 - recall: 0.8603 - val_loss: 2.3258 - val_accuracy: 0.7327 - val_precision: 0.7327 - val_recall: 1.0000
Epoch 2/6
261/261 [==============================] - 55s 212ms/step - loss: 0.2272 - accuracy: 0.9124 - precision: 0.9689 - recall: 0.9116 - val_loss: 4.2219 - val_accuracy: 0.2846 - val_precision: 1.0000 - val_recall: 0.0236
Epoch 3/6
261/261 [==============================] - 55s 210ms/step - loss: 0.1641 - accuracy: 0.9373 - precision: 0.9824 - recall: 0.9325 - val_loss: 1.8533 - val_accuracy: 0.4269 - val_precision: 1.0000 - val_recall: 0.2178
Epoch 4/6
261/261 [==============================] - 55s 210ms/step - loss: 0.1616 - accuracy: 0.9389 - precision: 0.9827 - recall: 0.9343 - val_loss: 9.2779 - val_accuracy: 0.3260 - val_precision: 1.0000 - val_recall: 0.0801
Epoch 5/6
261/261 [==============================] - 55s 209ms/step - loss: 0.1229 - a